In [38]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm010483",
    security_token="nc20DsgxOHWVR9OHZ78s7TRp",
)

Rescate de pacientes que no fueron creados por administradores

In [39]:
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name,HealthCloudGA__Gender__pc,CreatedById, CreatedDate, BillingCountry,Pa_s_del_Paciente__c, Phone  from Account where ispersonaccount = True and CreatedById <> '0058c00000Bpv8HAAR' and CreatedById <> '0058c00000972AcAAI' and CreatedById <> '0058c00000DSxZmAAL' and CreatedById <> '0058c00000DSxZrAAL' and CreatedById <> '0058c00000DT4SoAAL'
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "HealthCloudGA__Gender__pc",
        "CreatedById",
        "CreatedDate",
        "Phone",
        'BillingCountry',
        'Pa_s_del_Paciente__c'
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "CreatedById": "Creado por",
        "CreatedDate": "Fecha de Creacion",
        "HealthCloudGA__Gender__pc": "Sexo",
        'BillingCountry':"Pais 1",
        'Pa_s_del_Paciente__c':"Pais 2"
    }
)
Pacientes_SF.shape

(694, 9)

Recuperacion de usuarios del sistemas

In [40]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id,Name from user
    """
)
usuarios_SF = pd.DataFrame(results["records"])
usuarios_SF = usuarios_SF[
    [
        "Id",
        "Name",
    ]
]
usuarios_SF = usuarios_SF.rename(
    columns={
        "Id": "Id Account Usuario",        
        "Name": "Nombre Usuario",
        
    }
)
usuarios_SF.shape

(80, 2)

In [41]:
pacientes_usuarios = pd.merge(
    left=Pacientes_SF,
    right=usuarios_SF,
    how="left",
    left_on="Creado por",
    right_on="Id Account Usuario",
)
pacientes_usuarios.head()

,Id Account Paciente,Rut,Nombre,Sexo,Creado por,Fecha de Creacion,Phone,Pais 1,Pais 2,Id Account Usuario,Nombre Usuario
0,0018c00002amsahAAA,192380103,Sebastian Alejandro Fuentes Peña,Hombre,0058c00000DcdNfAAJ,2022-11-22T16:50:08.000+0000,945612300,None,a1U8c000005dAtnEAE,0058c00000DcdNfAAJ,Israel Medina Caneo
1,0018c00002anSzZAAU,189355742,María Jesus Ferrer Morey,Mujer,0058c00000DcdNTAAZ,2022-11-23T16:26:01.000+0000,None,Chile,a1U8c000005dAtnEAE,0058c00000DcdNTAAZ,Karen Castillo Morales
2,0018c00002anSzyAAE,154339302,María Feres Donoso,Mujer,0058c00000DcdNTAAZ,2022-11-23T16:30:31.000+0000,None,None,a1U8c000005dAtnEAE,0058c00000DcdNTAAZ,Karen Castillo Morales
3,0018c00002anT9AAAU,100662345,Inés Garrido Carrasco,Mujer,0058c00000DcdNTAAZ,2022-11-23T16:52:49.000+0000,None,None,a1U8c000005dAtnEAE,0058c00000DcdNTAAZ,Karen Castillo Morales
4,0018c00002anTPWAA2,195176493,Valentina Garay Vargas,Mujer,0058c00000DcdNTAAZ,2022-11-23T17:59:32.000+0000,None,None,a1U8c000005dAtnEAE,0058c00000DcdNTAAZ,Karen Castillo Morales


recuperar empresas

In [42]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,Vigencia__c, Corredor__c,Pa_s_del_Paciente__c from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "Vigencia__c",
        "Corredor__c",
        "Pa_s_del_Paciente__c",
    ]
]
Empresas_SF.shape

(6898, 10)

In [43]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape


(6880, 5)

In [44]:
# buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF.shape

(1188948, 4)

In [45]:
empresas_unidas = pd.merge(
    left=Programas_por_empresa_SF,
    right= Empresas_SF,
    how="left",
    left_on="Empresa__c",
    right_on="Id",
)
empresas_unidas.head()

,Id_x,Empresa__c,Empresa_Padre__c,Nombre_Empresa__c,Name_x,Id_y,Name_y,Holding__c,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,Vigencia__c,Corredor__c,Pa_s_del_Paciente__c
0,a1W8c000007c5aIEAQ,0018c00002k9KwBAAU,Aldo Conti,Aldo Conti MX,A-9930,0018c00002k9KwBAAU,Aldo Conti MX,None,Aldo Conti,EMP10376,Aldo Conti MX,None,SI,None,a1U8c000005dAtsEAE
1,a1W8c000007c669EAA,0018c00002k9yOCAAY,Vida Security,Caja De Compensación De Asignación De Los Andes,A-9931,0018c00002k9yOCAAY,Caja De Compensación De Asignación De Los Andes,Itaú,Vida Security,EMP10377,Caja De Compensación De Asignación De Los Andes,283144,SI,None,a1U8c000005dAtnEAE
2,a1W8c000007c66AEAQ,0018c00002k9yODAAY,Vida Security,Itaú Corpbanca,A-9932,0018c00002k9yODAAY,Itaú Corpbanca,Itaú,Vida Security,EMP10378,Itaú Corpbanca,283147,SI,None,a1U8c000005dAtnEAE
3,a1W8c000007c66BEAQ,0018c00002k9yOEAAY,Vida Security,Itaú Corpbanca,A-9933,0018c00002k9yOEAAY,Itaú Corpbanca,Itaú,Vida Security,EMP10379,Itaú Corpbanca,283148,SI,None,a1U8c000005dAtnEAE
4,a1W8c000007c66CEAQ,0018c00002k9yOFAAY,Vida Security,Itaú Corpbanca,A-9934,0018c00002k9yOFAAY,Itaú Corpbanca,Itaú,Vida Security,EMP10380,Itaú Corpbanca,283149,SI,None,a1U8c000005dAtnEAE


In [46]:
# Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=empresas_unidas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Id_x",
)
empresas_unidas.shape

(1188948, 19)

In [47]:
empresas_unidas.head(10)

,Id,Paciente__c,Programas_por_Empresas__c,padre__c,Id_x,Empresa__c,Empresa_Padre__c,Nombre_Empresa__c,Name_x,Id_y,Name_y,Holding__c,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,Vigencia__c,Corredor__c,Pa_s_del_Paciente__c
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,a1W8c000007vDPvEAM,0018c00002axQaeAAE,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,A-0173,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,EMP6008,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,SI,Directo BCH,a1U8c000005dAtnEAE
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,a1W8c000007vDSpEAM,0018c00002axQecAAE,MetLife,Clínica Alemana,A-0353,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,EMP6274,Clínica Alemana,340006119,SI,Willis Towers Watson S.A. Corred. De Seg.,a1U8c000005dAtnEAE
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,a1W8c000007vDSpEAM,0018c00002axQecAAE,MetLife,Clínica Alemana,A-0353,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,EMP6274,Clínica Alemana,340006119,SI,Willis Towers Watson S.A. Corred. De Seg.,a1U8c000005dAtnEAE
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,a1W8c000007vDSpEAM,0018c00002axQecAAE,MetLife,Clínica Alemana,A-0353,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,EMP6274,Clínica Alemana,340006119,SI,Willis Towers Watson S.A. Corred. De Seg.,a1U8c000005dAtnEAE
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,a1W8c000007vDSpEAM,0018c00002axQecAAE,MetLife,Clínica Alemana,A-0353,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,EMP6274,Clínica Alemana,340006119,SI,Willis Towers Watson S.A. Corred. De Seg.,a1U8c000005dAtnEAE
5,a1Y8c00000C5assEAB,0018c00002amLc3AAE,a1W8c000007vDSpEAM,MetLife,a1W8c000007vDSpEAM,0018c00002axQecAAE,MetLife,Clínica Alemana,A-0353,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,EMP6274,Clínica Alemana,340006119,SI,Willis Towers Watson S.A. Corred. De Seg.,a1U8c000005dAtnEAE
6,a1Y8c00000C5astEAB,0018c00002amLc8AAE,a1W8c000007vDSpEAM,MetLife,a1W8c000007vDSpEAM,0018c00002axQecAAE,MetLife,Clínica Alemana,A-0353,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,EMP6274,Clínica Alemana,340006119,SI,Willis Towers Watson S.A. Corred. De Seg.,a1U8c000005dAtnEAE
7,a1Y8c00000C5asuEAB,0018c00002amLcaAAE,a1W8c000007vDSpEAM,MetLife,a1W8c000007vDSpEAM,0018c00002axQecAAE,MetLife,Clínica Alemana,A-0353,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,EMP6274,Clínica Alemana,340006119,SI,Willis Towers Watson S.A. Corred. De Seg.,a1U8c000005dAtnEAE
8,a1Y8c00000C5asvEAB,0018c00002amLccAAE,a1W8c000007vDSpEAM,MetLife,a1W8c000007vDSpEAM,0018c00002axQecAAE,MetLife,Clínica Alemana,A-0353,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,EMP6274,Clínica Alemana,340006119,SI,Willis Towers Watson S.A. Corred. De Seg.,a1U8c000005dAtnEAE
9,a1Y8c00000C5aswEAB,0018c00002amLcdAAE,a1W8c000007vDSpEAM,MetLife,a1W8c000007vDSpEAM,0018c00002axQecAAE,MetLife,Clínica Alemana,A-0353,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,EMP6274,Clínica Alemana,340006119,SI,Willis Towers Watson S.A. Corred. De Seg.,a1U8c000005dAtnEAE


In [48]:
pacientes_usuarios['check'] = pacientes_usuarios['Id Account Paciente'].isin(empresas_unidas['Paciente__c'])

In [49]:
pacientes_usuarios.to_excel("E:/Siempre/Usuarios/Francisca Rubio Yáñez/TI/BBDD/usuarios_sin empresas.xlsx", index=False)
pacientes_usuarios.head()

,Id Account Paciente,Rut,Nombre,Sexo,Creado por,Fecha de Creacion,Phone,Pais 1,Pais 2,Id Account Usuario,Nombre Usuario,check
0,0018c00002amsahAAA,192380103,Sebastian Alejandro Fuentes Peña,Hombre,0058c00000DcdNfAAJ,2022-11-22T16:50:08.000+0000,945612300,None,a1U8c000005dAtnEAE,0058c00000DcdNfAAJ,Israel Medina Caneo,False
1,0018c00002anSzZAAU,189355742,María Jesus Ferrer Morey,Mujer,0058c00000DcdNTAAZ,2022-11-23T16:26:01.000+0000,None,Chile,a1U8c000005dAtnEAE,0058c00000DcdNTAAZ,Karen Castillo Morales,True
2,0018c00002anSzyAAE,154339302,María Feres Donoso,Mujer,0058c00000DcdNTAAZ,2022-11-23T16:30:31.000+0000,None,None,a1U8c000005dAtnEAE,0058c00000DcdNTAAZ,Karen Castillo Morales,True
3,0018c00002anT9AAAU,100662345,Inés Garrido Carrasco,Mujer,0058c00000DcdNTAAZ,2022-11-23T16:52:49.000+0000,None,None,a1U8c000005dAtnEAE,0058c00000DcdNTAAZ,Karen Castillo Morales,True
4,0018c00002anTPWAA2,195176493,Valentina Garay Vargas,Mujer,0058c00000DcdNTAAZ,2022-11-23T17:59:32.000+0000,None,None,a1U8c000005dAtnEAE,0058c00000DcdNTAAZ,Karen Castillo Morales,True
